In [1]:
import numpy as np 
import shutil # pour les dossiers
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
def metrique(path):
    files = os.listdir(path);
    contenu = [];
    for file in files:
        if file.endswith('.json'):
            openFile = open(path+file, "r");
            contenu.append(json.loads(openFile.read())["Image"]);
            openFile.close();
    return contenu;

In [3]:
chemin = "./train/";
contenu = [];
if not os.path.isdir(chemin):
    print('Rajouter le dossier train dans le dossier courant :) ! ');
else :
    contenu = metrique(chemin);
    df = pd.DataFrame(contenu)
    # on laisse Species car c'est égale au nombre de classe
    df.drop(["Genus","ClassId","Family","Vote","Location","Latitude","Longitude","Date","Author","Content","MediaId", "LearnTag", "ImageId2014","ObservationId2014","YearInCLEF","ObservationId"], axis='columns', inplace=True)
    df = df.drop_duplicates()
    classe  = df.to_numpy();
    print(len(classe))


50


In [4]:
X = [];
Y = [];
files = os.listdir("./train")
for file in files:
    if file.endswith('.jpg'):
        X.append(file);
        nameFile = file.split('.')[0];
        for file_class in files:
            nameFileAutre = file_class.split('.')[0];
            if nameFile==nameFileAutre and file_class.endswith('.json'):
                fichierSrc = open("./train/"+file_class, "r");
                contenu = fichierSrc.read();
                monJson = json.loads(contenu)
                Y.append(monJson["Image"]["Species"])
                break;


In [5]:
# on a bien une classe par image !
if(len(X) == len(Y)):
    print("OK")
else:
    print("KO")

OK


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, stratify=Y)
y_train[0]

'Salix caprea L.'

In [7]:
# Création des répertoires ! 
directory = "MLP"
for classe in y_train : 
    if(not os.path.isdir('./MLP/train/'+classe)):
        os.mkdir('./MLP/train/'+classe)
        
for classe in y_test : 
    if(not os.path.isdir('./MLP/val/'+classe)):
        os.mkdir('./MLP/val/'+classe)
        
# on met les images dans les répertoires precédents

# on supprime le contenu

directorys=os.listdir('./MLP/train/')
for i in range(0,len(directory)):
    files=os.listdir('./MLP/train/'+directorys[i])
    for j in range(0,len(files)):
        os.remove('./MLP/train/'+directorys[i]+'/'+files[j])

directorys=os.listdir('./MLP/val/')
for i in range(0,len(directory)):
    files=os.listdir('./MLP/val/'+directorys[i])
    for j in range(0,len(files)):
        os.remove('./MLP/val/'+directorys[i]+'/'+files[j])

# on met le contenu
shutil.copyfile('./train/'+X_train[0],'./MLP/train/'+y_train[0]+'/'+X_train[0])

print('./MLP/train/'+y_train[0]+X_train[0])
print(X_train[0])
for i in range(len(X_train)):
    shutil.copyfile('./train/'+X_train[i],'./MLP/train/'+y_train[i]+'/'+X_train[i])
    
for i in range(len(X_test)):
    shutil.copyfile('./train/'+X_test[i],'./MLP/val/'+y_test[i]+'/'+X_test[i])
    

./MLP/train/Salix caprea L.30108.jpg
30108.jpg


In [89]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = 'MLP'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes
#print(pd.DataFrame(class_names))



In [90]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(3*224*224,128), # (input_size = batch_size * image_size,hidden_layers)
            nn.ReLU(),
            nn.Linear(128,64) # (hidden_layers,num_classes)
        )
        
    def forward(self,x):
        x=x.view(x.size(0),-1)
        x=self.layers(x)
        return x

In [91]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [92]:
model = MLP()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()

mean_train_losses = []
mean_valid_losses = []
epochs = 15

for epoch in range(epochs):
    
    train_losses=[]
    valid_losses=[]
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
            for images,labels in dataloaders[phase]:                
                optimizer.zero_grad()
                outputs = model(images)
                loss = loss_fn(outputs,labels)
                loss.backward()
                optimizer.step()
            
                train_losses.append(loss.item())
                            
            model.eval()
            correct = 0
            total = 0
        else:
            with torch.no_grad():   # Set model to evaluate mode
                for i, (images,labels) in dataloaders[phase]:
                    outputs = model(images)
                    loss = loss_fn(outputs,labels)
                    
                    valid_losses.append(loss.item())
                    
                    _, predicted = torch.max(outputs.data, 1)
                    correct += (predicted == labels).sum().item()
                    total += labels.size(0)
            mean_train_losses.append(np.mean(train_losses))
            mean_valid_losses.append(np.mean(valid_losses))
            
            accuracy = 100*correct/total
            valid_acc_list.append(accuracy)
            print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}, valid acc : {:.2f}%'\
         .format(epoch+1, np.mean(train_losses), np.mean(valid_losses), accuracy))

ValueError: too many values to unpack (expected 2)

In [93]:
print(image_datasets['train'])

Dataset ImageFolder
    Number of datapoints: 2327
    Root location: MLP/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [94]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    evolution_accuracy = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            evolution_accuracy.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print("Evolution de l'accuracy pour savoir si on doit augmenter l'époch : ")
    plt.plot(evolution_accuracy)
    plt.ylabel('Evolution accuracy par epoch')
    plt.show()

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [95]:
model_ft = MLP()
#num_ftrs = model_ft.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
class_names = image_datasets['train'].classes
#model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
#torch.optim.Adam(model.parameters(),lr=0.001)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

best_model = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=20)

Epoch 0/19
----------
train Loss: 4.2163 Acc: 0.0138
val Loss: 4.2187 Acc: 0.0113

Epoch 1/19
----------
train Loss: 4.2131 Acc: 0.0125
val Loss: 4.2187 Acc: 0.0113

Epoch 2/19
----------
train Loss: 4.2091 Acc: 0.0142
val Loss: 4.2187 Acc: 0.0113

Epoch 3/19
----------
train Loss: 4.2094 Acc: 0.0120
val Loss: 4.2187 Acc: 0.0113

Epoch 4/19
----------
train Loss: 4.2129 Acc: 0.0107
val Loss: 4.2187 Acc: 0.0113

Epoch 5/19
----------
train Loss: 4.2223 Acc: 0.0107
val Loss: 4.2187 Acc: 0.0113

Epoch 6/19
----------
train Loss: 4.2142 Acc: 0.0120
val Loss: 4.2187 Acc: 0.0113

Epoch 7/19
----------
train Loss: 4.2118 Acc: 0.0133
val Loss: 4.2187 Acc: 0.0113

Epoch 8/19
----------
train Loss: 4.2120 Acc: 0.0120
val Loss: 4.2187 Acc: 0.0113

Epoch 9/19
----------


KeyboardInterrupt: 

In [48]:
for parameter in model.parameters():
    print("One param")
    print(parameter)

One param
Parameter containing:
tensor([[-0.4789,  0.4410,  0.2044, -0.2410],
        [ 0.3769,  0.1502, -0.0799, -0.3023],
        [-0.4483,  0.2311, -0.2060,  0.0248],
        ...,
        [ 0.1169,  0.4867, -0.0776,  0.2218],
        [ 0.0521, -0.0844,  0.4051,  0.3249],
        [-0.3128,  0.0269, -0.1078,  0.1951]], requires_grad=True)
One param
Parameter containing:
tensor([ 4.6833e-01, -2.5645e-01, -1.1559e-02,  3.0567e-01, -2.5046e-01,
         1.9443e-02, -1.0949e-01, -2.4415e-01,  4.8110e-02, -3.3466e-01,
         3.5572e-01,  2.1433e-01, -4.7692e-01, -4.0853e-01,  3.1212e-01,
        -1.6819e-02,  1.3692e-01,  4.4063e-01, -4.6919e-02, -1.4800e-01,
        -4.0496e-02, -4.1938e-01,  1.6247e-01,  3.1832e-01,  2.6616e-01,
         3.5101e-01, -2.6829e-01,  1.2898e-01, -4.0994e-01, -3.0156e-01,
         6.8122e-02, -3.2371e-01,  1.0602e-01,  3.2798e-01,  3.4014e-01,
        -4.2570e-02,  1.6644e-01,  3.3812e-01,  2.4287e-01,  1.1193e-01,
         1.2993e-01,  5.0408e-02, -7.2067e